In [16]:
import pandas as pd
from pathlib import Path

data_dir = Path("../data")

files = sorted(data_dir.glob("ESMGFZ_AAM_v1.0_03h_*.asc.csv"))
len(files), files[:3], files[-3:]

def load_one_year_aam(path: Path) -> pd.DataFrame:
    df = pd.read_csv(path, sep=";")

    # Build datetime index (Time is like "03:00:00")
    df["datetime"] = (
        pd.to_datetime(dict(year=df["Year"], month=df["Month"], day=df["Day"]))
        + pd.to_timedelta(df["Time"])   # <-- no unit=
    )

    # Axial atmospheric excitation
    df["X3_atm"] = (
        pd.to_numeric(df["Mass_Z"], errors="coerce") +
        pd.to_numeric(df["Motion_Z"], errors="coerce")
    )

    return df[["datetime", "X3_atm"]].dropna()


frames = [load_one_year_aam(f) for f in files]
aam_all = (
    pd.concat(frames, ignore_index=True)
      .sort_values("datetime")
      .set_index("datetime")
)

aam_monthly = aam_all["X3_atm"].resample("MS").mean()
aam_monthly.describe()

# aam_all.head(), aam_all.tail(), aam_all.shape


count    6.000000e+02
mean     7.257515e-10
std      4.164518e-09
min     -1.069905e-08
25%     -2.536401e-09
50%      1.656963e-09
75%      3.843782e-09
max      9.782018e-09
Name: X3_atm, dtype: float64